## Run inference on the Llama 2 endpoint you have created.

In [ ]:
import json
import boto3

### Supported Parameters

***
This model supports many parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches `max_new_tokens`. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.
* **stop**: If specified, it must be a list of strings. Text generation stops if any one of the specified strings is generated.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments.

**NOTE**: If `max_new_tokens` is not defined, the model may generate up to the maximum total tokens allowed, which is 4K for these models. This may result in endpoint query timeout errors, so it is recommended to set `max_new_tokens` when possible. For 7B, 13B, and 70B models, we recommend to set `max_new_tokens` no greater than 1500, 1000, and 500 respectively, while keeping the total number of tokens less than 4K.

***

In [3]:
zero_shot_prompts = [
    "I believe the meaning of life is",
    "Simply put, the theory of relativity states that ",
    """A brief message congratulating the team on the launch:

Hi everyone,

I just """,
]
few_shot_prompts = [
    """Translate English to French:
sea otter => loutre de mer
peppermint => menthe poivrée
plush girafe => girafe peluche
cheese =>""",
]

payloads = []
for prompt in zero_shot_prompts:
    payloads.append(
        {
            "inputs": prompt, 
            "parameters": {"max_new_tokens": 64, "top_p": 0.9, "temperature": 0.6, "return_full_text": False},
        }
    )
for prompt in few_shot_prompts:
    payloads.append(
        {
            "inputs": prompt, 
            "parameters": {"max_new_tokens": 64, "top_p": 0.9, "temperature": 0.6, "return_full_text": False},
        }
    )

### Query endpoint that you have created

---

In [102]:
endpoint_name = 'jumpstart-dft-meta-textgeneration-llama-2-7b'


def query_endpoint(payload):
    client = boto3.client("sagemaker-runtime")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
    )
    #client.delete_endpoint(EndpointName=endpoint_name)
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response

In [103]:
for payload in payloads:
    query_response = query_endpoint(payload)
    print(payload["inputs"])
    print(f"> {query_response[0]['generated_text']}")
    print("\n======\n")

I believe the meaning of life is
>  to find out what you love, and then to do it.
I love the outdoors, and I love to be creative. So, I decided to combine my two passions, and I became a photographer.
I love to be outdoors, and I love to be creative.


Simply put, the theory of relativity states that 
> 1) the laws of physics are the same for all observers, regardless of their state of motion and 2) the speed of light is the same for all observers, regardless of their state of motion. The theory of relativity is one of the most important scientific discoveries of all time, and it has


A brief message congratulating the team on the launch:

Hi everyone,

I just 
> wanted to say a huge thank you for all of your hard work on the project. The launch went incredibly well and we are so pleased with how it all turned out.

I look forward to working with you all again soon.

All the best,

Sarah



Translate English to French:
sea otter => loutre de mer
peppermint => menthe poivrée
plush gira

In [11]:
# Optional: update pip
%pip install pip -Uq

# Install latest versions of other required packages
%pip install ipywidgets langchain awscli boto3 python-dotenv SQLAlchemy psycopg2-binary chromadb -Uq
%pip install pyyaml -q


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 

In [10]:
%pip list | grep 'langchain\|sentence-transformers\|SQLAlchemy'

SQLAlchemy                           1.4.22

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [104]:
import langchain
langchain.debug = True

In [105]:
import json
Q1 = "How many unique teams are there?"

In [106]:
REGION_NAME = "us-east-1"

In [274]:
from langchain.llms.sagemaker_endpoint import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler


class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        print(response_json)
        return response_json[0]["generated_text"]#[0]


content_handler = ContentHandler()

parameters = {
    "max_length": 2000,
   # "temperature": 0.0,
    "top_p":1,
}

# smaller model endpoint
llm_sml = SagemakerEndpoint(
    endpoint_name=endpoint_name,
    region_name=REGION_NAME,
    model_kwargs=parameters,
    content_handler=content_handler,
)

# larger model endpoint
llm_lrg = SagemakerEndpoint(
    endpoint_name=endpoint_name,
    region_name=REGION_NAME,
    model_kwargs=parameters,
    content_handler=content_handler,
)

In [108]:
#!pip install langchain langchain_experimental

In [109]:
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_experimental.sql import SQLDatabaseSequentialChain

In [110]:
import json
Q1 = "How many unique teams are there?"

In [184]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM artists LIMIT 10;")

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [127]:
db.run("pragma table_info(employees);")

"[(0, 'EmployeeId', 'INTEGER', 1, None, 1), (1, 'LastName', 'NVARCHAR(20)', 1, None, 0), (2, 'FirstName', 'NVARCHAR(20)', 1, None, 0), (3, 'Title', 'NVARCHAR(30)', 0, None, 0), (4, 'ReportsTo', 'INTEGER', 0, None, 0), (5, 'BirthDate', 'DATETIME', 0, None, 0), (6, 'HireDate', 'DATETIME', 0, None, 0), (7, 'Address', 'NVARCHAR(70)', 0, None, 0), (8, 'City', 'NVARCHAR(40)', 0, None, 0), (9, 'State', 'NVARCHAR(40)', 0, None, 0), (10, 'Country', 'NVARCHAR(40)', 0, None, 0), (11, 'PostalCode', 'NVARCHAR(10)', 0, None, 0), (12, 'Phone', 'NVARCHAR(24)', 0, None, 0), (13, 'Fax', 'NVARCHAR(24)', 0, None, 0), (14, 'Email', 'NVARCHAR(60)', 0, None, 0)]"

In [126]:
db.run("select * from employees")

"[(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com'), (2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com'), (3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com'), (4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com'), (5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgar

In [119]:
from langchain.chains import create_sql_query_chain

In [134]:
chain = create_sql_query_chain(llm_lrg, db)
response = chain.invoke({"question": "How many employees are there"})
chain.get_prompts()[0].pretty_print()

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "How many employees are there"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info>] Entering Chain run with input:
{
  "question": "How many employees are there"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info>] Entering Chain run with input:
{
  "question": "How many employees are there"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info> > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "How many employees are there"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info> > 4:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "How many employees are there\nSQLQuery: "
}
[chain/start] [1:chain:Runn

In [135]:
db.run(response)

OperationalError: (sqlite3.OperationalError) near "```
SELECT COUNT(DISTINCT ReportsTo) AS "Number of Employees" 
FROM employees
```": syntax error
[SQL: ```
SELECT COUNT(DISTINCT ReportsTo) AS "Number of Employees" 
FROM employees
```]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [141]:
chain2 = create_sql_query_chain(llm_lrg, db)
response2 = chain.invoke({"question": "Use table Employees. How many employees are there"})
chain2.get_prompts()[0].pretty_print()

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Use table Employees. How many employees are there"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info>] Entering Chain run with input:
{
  "question": "Use table Employees. How many employees are there"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info>] Entering Chain run with input:
{
  "question": "Use table Employees. How many employees are there"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info> > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "Use table Employees. How many employees are there"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info> > 4:chain:RunnableLambda] s] Exiting Chain run with output:
{


In [142]:
db.run(response2)

'[(8,)]'

In [143]:
response2

'SELECT COUNT(DISTINCT "EmployeeId") AS "Question: Employees. How many employees are there"\nFROM "Employees"'

In [144]:
print(db.dialect)
print(db.get_usable_table_names())
#db.run("SELECT * FROM artists LIMIT 10;")

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


In [147]:
db.run("select * from artists")

'[(1, \'AC/DC\'), (2, \'Accept\'), (3, \'Aerosmith\'), (4, \'Alanis Morissette\'), (5, \'Alice In Chains\'), (6, \'Antônio Carlos Jobim\'), (7, \'Apocalyptica\'), (8, \'Audioslave\'), (9, \'BackBeat\'), (10, \'Billy Cobham\'), (11, \'Black Label Society\'), (12, \'Black Sabbath\'), (13, \'Body Count\'), (14, \'Bruce Dickinson\'), (15, \'Buddy Guy\'), (16, \'Caetano Veloso\'), (17, \'Chico Buarque\'), (18, \'Chico Science & Nação Zumbi\'), (19, \'Cidade Negra\'), (20, \'Cláudio Zoli\'), (21, \'Various Artists\'), (22, \'Led Zeppelin\'), (23, \'Frank Zappa & Captain Beefheart\'), (24, \'Marcos Valle\'), (25, \'Milton Nascimento & Bebeto\'), (26, \'Azymuth\'), (27, \'Gilberto Gil\'), (28, \'João Gilberto\'), (29, \'Bebel Gilberto\'), (30, \'Jorge Vercilo\'), (31, \'Baby Consuelo\'), (32, \'Ney Matogrosso\'), (33, \'Luiz Melodia\'), (34, \'Nando Reis\'), (35, \'Pedro Luís & A Parede\'), (36, \'O Rappa\'), (37, \'Ed Motta\'), (38, \'Banda Black Rio\'), (39, \'Fernanda Porto\'), (40, \'Os Ca

In [174]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_types import AgentType
#chain3a = create_sql_agent(llm_lrg, db, agent_type= AgentType.ZERO_SHOT_REACT_DESCRIPTION)
#response3a = chain3a.invoke({"question": "Use table artists. How many artists are there"})
#chain3a.get_prompts()[0].pretty_print()

In [173]:
response3b

'SELECT \n\tname\nFROM p_alertusers\nORDER BY\n\tname;'

In [183]:
db

In [ ]:
create_sql_query_chain

In [185]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm_lrg,db = db,agent_type="zero-shot-react-description", verbose = True)

In [186]:
agent_executor.invoke({"input":"Describe the schema of the playlist table"})

[chain/start] [1:chain:SQL Agent Executor] Entering Chain run with input:
{
  "input": "Describe the schema of the playlist table"
}
[chain/start] [1:chain:SQL Agent Executor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:SQL Agent Executor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:SQL Agent Executor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:SQL Agent Executor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [1:chain:SQL Agent Executor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scra

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: ` Depending on how many data you want, you should either manually unpack the json file or read it using the bkb_read tool
Action: You should use the bkb_pull tool to retrieve your elasticsearch results in the easily usable format
Action Input is a string like:

```
"http://es-127:9200/bkb_backend/_search?scroll=1m&pretty": {
  "query": {
`

In [181]:
#agent_executor.invoke("List the total sales per country. Which country's customers spent the most?")

In [190]:
agent_executor2 = create_sql_agent(llm_lrg,db = db,agent_type="zero-shot-react-description", verbose = False)
#agent_executor2.invoke("How many employees are there?")

In [187]:
agent_executor.invoke("How many employees are there?")

[chain/start] [1:chain:SQL Agent Executor] Entering Chain run with input:
{
  "input": "How many employees are there?"
}
[chain/start] [1:chain:SQL Agent Executor > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:SQL Agent Executor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:SQL Agent Executor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [1:chain:SQL Agent Executor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [1:chain:SQL Agent Executor > 2:chain:RunnableSequence > 3:chain:RunnableAssign<agent_scratchpad> > 4:chain:RunnableParallel<agent_scratchpad> > 5:

{'input': 'How many employees are there?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [172]:
chain3b = create_sql_query_chain(llm_lrg, db)
response3b = chain3b.invoke({"question": "what is?"})
chain3b.get_prompts()[0].pretty_print()

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "what is my name?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info>] Entering Chain run with input:
{
  "question": "what is my name?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info>] Entering Chain run with input:
{
  "question": "what is my name?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info> > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "what is my name?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info> > 4:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "what is my name?\nSQLQuery: "
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:

In [157]:
response3b

'SELECT first_name(albums.fname) || ‘ ’'

In [156]:
db.run(response3b)

OperationalError: (sqlite3.OperationalError) no such column: ‘
[SQL: SELECT first_name(albums.fname) || ‘ ’]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [208]:
def clearMemory():
    conversation.memory.clear()

In [209]:
clearMemory()

NameError: name 'conversation' is not defined

In [387]:
response3d

"SELECT DISTINCT tracks.AlbumId, tracks.MediaTypeId, tracks.Composer, tracks.Milliseconds, tracks.Bytes\nFROM ALLTABLES \n--exclude customers, invoices, playlists, artists, genres\nWHERE artists.Name = 'AC/DC' \nAND albums.Title = 'Out Of Exile' \n--AND invoices.UnitPrice <= 2 \n--AND songs.Quantity >"

In [383]:
#db.run("select * from albums")

In [385]:
#db.run(response3d)

In [254]:
#llm_sml.delete_endpoint()

In [386]:
chain3d = create_sql_query_chain(llm_lrg, db)
#response3d = chain3d.invoke({"question": "Use table artists. How many artists are there"})
#response3d = chain3d.invoke({"question": "Use table albums. How many albums are there"})
#\'Out Of Exile\'
response3d = chain3d.invoke({"question": "Use tables albums. what is the artistid for Title \'Out Of Exile\' "})

chain3d.get_prompts()[0].pretty_print()

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Use tables albums. what is the artistid for Title 'Out Of Exile' "
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info>] Entering Chain run with input:
{
  "question": "Use tables albums. what is the artistid for Title 'Out Of Exile' "
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info>] Entering Chain run with input:
{
  "question": "Use tables albums. what is the artistid for Title 'Out Of Exile' "
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info> > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "Use tables albums. what is the artistid for Title 'Out Of Exile' "
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info

In [148]:
chain3 = create_sql_query_chain(llm_lrg, db)
response3 = chain.invoke({"question": "Use table artists. How many artists are there"})
chain3.get_prompts()[0].pretty_print()

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Use table artists. How many artists are there"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info>] Entering Chain run with input:
{
  "question": "Use table artists. How many artists are there"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info>] Entering Chain run with input:
{
  "question": "Use table artists. How many artists are there"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info> > 4:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "Use table artists. How many artists are there"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableAssign<input,table_info> > 3:chain:RunnableParallel<input,table_info> > 4:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "Use

In [149]:
db.run(response3)

'[(275,)]'

In [112]:
# Reference: https://python.langchain.com/en/latest/modules/chains/examples/sqlite.html#sqldatabasesequentialchain
from sqlalchemy.exc import ProgrammingError


#db = SQLDatabase.from_uri("sqlite:///nba_roster.db")
#db = SQLDatabase.from_uri("sqlite:///nba_roster1.db", sample_rows_in_table_info= 0,use_query_checker)

db_chain = SQLDatabaseSequentialChain.from_llm(
    llm_lrg, db, verbose=True, use_query_checker=True
)

try:
    db_chain(Q1)
except (ProgrammingError, ValueError) as exc:
    print(f"\n\n{exc}")


[chain/start] [1:chain:SQLDatabaseSequentialChain] Entering Chain run with input:
{
  "query": "How many unique teams are there?"
}
[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "query": "How many unique teams are there?",
  "table_names": "nba_roster"
}
[llm/start] [1:chain:LLMChain > 2:llm:SagemakerEndpoint] Entering LLM run with input:
{
  "prompts": [
    "Given the below input question and list of potential tables, output a comma separated list of the table names that may be necessary to answer this question.\n\nQuestion: How many unique teams are there?\n\nTable Names: nba_roster\n\nRelevant Table Names:"
  ]
}


/opt/conda/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[{'generated_text': '\n\nnba_roster\n\nstats [2 fields:] cmp, points - per table\n\nEPL can be confusing because each record may have more than one name.'}]
[llm/end] [1:chain:LLMChain > 2:llm:SagemakerEndpoint] [1.29s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n\nnba_roster\n\nstats [2 fields:] cmp, points - per table\n\nEPL can be confusing because each record may have more than one name.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:LLMChain] [1.29s] Exiting Chain run with output:
{
  "table_names": "\n\nnba_roster\n\nstats [2 fields:] cmp, points - per table\n\nEPL can be confusing because each record may have more than one name."
}
[chain/start] [1:chain:SQLDatabaseSequentialChain > 2:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "How many unique teams are there?",
  "table_names_to_use": []
}
[chain/start] [1:chain:SQLDataba

OperationalError: (sqlite3.OperationalError) near "'''

CREATE TABLE IF NOT EXISTS `team` (
`id` text NOT NULL,
`assno1` text NOT NULL,
`assno2` text NOT NULL
) ENGINE=InnoDB DEFAULT CHARSET=latin1;

INSERT INTO `team` (
`assno1`,
`assno2`
) VALUES ('": syntax error
[SQL: '''

CREATE TABLE IF NOT EXISTS `team` (
`id` text NOT NULL,
`assno1` text NOT NULL,
`assno2` text NOT NULL
) ENGINE=InnoDB DEFAULT CHARSET=latin1;

INSERT INTO `team` (
`assno1`,
`assno2`
) VALUES ('A 123', 'B 234');

CREATE TABLE IF NOT]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
memory = ConversationBufferMemory()

In [75]:
import json

In [29]:
PROMPT_SUFFIX = """
Only use the following tables:
{table_info}

Question: {input}"""

In [42]:
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=PROMPT_SUFFIX
)
db_chain2 = db_chain = SQLDatabaseSequentialChain.from_llm(
    llm_lrg, db, verbose=True, use_query_checker=True
)
db_chain2("How many unique teams are there?")


[chain/start] [1:chain:SQLDatabaseSequentialChain] Entering Chain run with input:
{
  "query": "How many unique teams are there?"
}
[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "query": "How many unique teams are there?",
  "table_names": ""
}
[llm/start] [1:chain:LLMChain > 2:llm:SagemakerEndpoint] Entering LLM run with input:
{
  "prompts": [
    "Given the below input question and list of potential tables, output a comma separated list of the table names that may be necessary to answer this question.\n\nQuestion: How many unique teams are there?\n\nTable Names: \n\nRelevant Table Names:"
  ]
}
[llm/error] [1:chain:LLMChain > 2:llm:SagemakerEndpoint] s] LLM run errored with error:
"ValueError('Error raised by inference endpoint: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary with message \"Failed to deserialize the JSON body into the target type: missing field `inputs` at line 1 column 301\". S

/opt/conda/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


ValueError: Error raised by inference endpoint: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary with message "Failed to deserialize the JSON body into the target type: missing field `inputs` at line 1 column 301". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/jumpstart-dft-meta-textgeneration-llama-2-7b in account 590116326836 for more information.

In [30]:
db_chain_memory = SQLDatabaseChain.from_llm(llm_lrg, db, memory=memory, 
                                            verbose=True, return_sql=True, 
                                            prompt=PromptTemplate(input_variables=["input", "table_info"], 
                                            template=PROMPT_SUFFIX))

In [31]:
# first question
db_chain_memory.run("How many unique teams are there?")

[chain/start] [1:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "How many unique teams are there?",
  "history": ""
}
[chain/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "How many unique teams are there?\nSQLQuery:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "",
  "stop": [
    "\nSQLResult:"
  ],
  "history": ""
}
[llm/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain > 3:llm:SagemakerEndpoint] Entering LLM run with input:
{
  "prompts": [
    "Only use the following tables:\n\n\nQuestion: How many unique teams are there?\nSQLQuery:"
  ]
}
[llm/error] [1:chain:SQLDatabaseChain > 2:chain:LLMChain > 3:llm:SagemakerEndpoint] s] LLM run errored with error:
"ValueError('Error raised by inference endpoint: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary with message \"Failed to deserialize the JSON body into the target type: missing field

ValueError: Error raised by inference endpoint: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary with message "Failed to deserialize the JSON body into the target type: missing field `inputs` at line 1 column 148". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/jumpstart-dft-meta-textgeneration-llama-2-7b in account 590116326836 for more information.